# Data Definition Language - DDL
Let us create a database `post`

In [1]:
%load_ext sql
%sql hive://hadoop@localhost:10000/

In [2]:
%%sql
CREATE DATABASE IF NOT EXISTS post

 * hive://hadoop@localhost:10000/
Done.


[]

In [3]:
%sql USE post

 * hive://hadoop@localhost:10000/
Done.


[]

## PLZ Verzeichnis

Let's focus first on the PLZ dataset

In [4]:
!head -n3 /data/dataset/post/plz_verzeichnis_v2.csv

REC_ART;ONRP;BFSNR;PLZ_TYP;POSTLEITZAHL;PLZ_ZZ;GPLZ;ORTBEZ18;ORTBEZ27;KANTON;SPRACHCODE;SPRACHCODE_ ABW;BRIEFZ_DURCH;GILT_AB_DAT;PLZ_BRIEFZUST;PLZ_COFF;Geo Shape;Geokoordinaten
01;111;5586;80;1000;07;1000;Lausanne St-Paul;Lausanne St-Paul;VD;2;;130;1993-09-28;100060;;;
01;118;5586;80;1000;17;1000;Lausanne 17;Lausanne 17;VD;2;;130;1986-05-21;100060;;;


### We see that

|REC_ART|ONRP|BFSNR|PLZ_TYP|POSTLEITZAHL|PLZ_ZZ|GPLZ|ORTBEZ18|ORTBEZ27|KANTON|SPRACHCODE|SPRACHCODE_ ABW|BRIEFZ_DURCH|GILT_AB_DAT|PLZ_BRIEFZUST|PLZ_COFF|Geo Shape|Geokoordinaten|
| ------- | ------- | ------- | ------- | ------- | ------- | ------- | ------- | ------- | ------- | ------- | ------- | ------- | ------- | ------- | ------- | ------- | ------- |
|01|111|5586|80|1000|07|1000|Lausanne St-Paul|Lausanne St-Paul|VD|2||130|1993-09-28|100060||||


1. The separator is a `;`.
2. "Kanton" would be a great partition. We will create one when we convert it to parquet.


Remember, the different types `hive` can use are [here](https://cwiki.apache.org/confluence/pages/viewpage.action?pageId=82706456#LanguageManualTypes-date).

Die Post gave additional information on the entire dataset [here](https://swisspost.opendatasoft.com/api/datasets/1.0/plz_verzeichnis_v2/attachments/strassenverzeichnis_mit_sortierdaten_de_pdf/).

E.g. for the PLZ dataset:

| Field name | Field type (length) | Mandatory field | Source | Observations |
| ------ | ------- | ------ | ------- | ------ |
| REC_ART | VARCHAR(2) | YES | “01” |Record type: Designates the record type. |
| ONRP | NUMBER(5) | YES | ASDP | Swiss Post classification number: This number (ONRP) is the primary key designating a postcode/location in accordance with the Swiss Post postcode database and the unique, unalterable key term of the postcode. |
| BFSNR | NUMBER(5) | YES | ASDP | Foreign key for BFSNR (refers to NEW_COM)|

etc.

In [5]:
%%sql
CREATE TABLE IF NOT EXISTS plz_csv (
    REC_ART VARCHAR(2),
    ONRP INT,
    BFSNR INT,
    PLZ_TYP SMALLINT,
    POSTLEITZAHL SMALLINT,
    PLZ_ZZ VARCHAR(2), 
    GPLZ SMALLINT,
    ORTBEZ18 VARCHAR(18),
    ORTBEZ27 VARCHAR(27),
    KANTON VARCHAR(2),
    SPRACHCODE TINYINT,
    SPRACHCODE_ABW TINYINT,
    BRIEFZ_DURCH INT,
    GILT_AB_DAT DATE,
    PLZ_BRIEFZUST INT,
    PLZ_COFF VARCHAR(1),
    Geo_Shape STRING,
    Geokoordinaten STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ';'
LINES TERMINATED BY '\n'
STORED AS TEXTFILE
tblproperties ("skip.header.line.count"="1")

 * hive://hadoop@localhost:10000/
Done.


[]

In [6]:
%%sql
LOAD DATA LOCAL INPATH '/data/dataset/post/plz_verzeichnis_v2.csv' INTO TABLE plz_csv

 * hive://hadoop@localhost:10000/
Done.


[]

In [7]:
%%sql 
SELECT
    REC_ART,
    ONRP,
    BFSNR,
    PLZ_TYP,
    POSTLEITZAHL,
    PLZ_ZZ, 
    GPLZ,
    ORTBEZ18,
    ORTBEZ27,
    SPRACHCODE,
    SPRACHCODE_ABW,
    BRIEFZ_DURCH,
    GILT_AB_DAT,
    PLZ_BRIEFZUST,
    PLZ_COFF,
    Kanton
from plz_csv limit 10

 * hive://hadoop@localhost:10000/
Done.


rec_art,onrp,bfsnr,plz_typ,postleitzahl,plz_zz,gplz,ortbez18,ortbez27,sprachcode,sprachcode_abw,briefz_durch,gilt_ab_dat,plz_briefzust,plz_coff,kanton
01,111,5586,80,1000,07,1000,Lausanne St-Paul,Lausanne St-Paul,2,None,130,1993-09-28,100060,,VD
01,118,5586,80,1000,17,1000,Lausanne 17,Lausanne 17,2,None,130,1986-05-21,100060,,VD
01,119,5586,80,1000,19,1000,Lausanne 19,Lausanne 19,2,None,130,1993-09-28,100060,,VD
01,120,5586,80,1000,20,1000,Lausanne Sévelin,Lausanne Sévelin,2,None,130,1993-09-06,100060,,VD
01,125,5586,20,1000,25,1000,Lausanne 25,Lausanne 25,2,None,130,1988-03-01,100060,J,VD
01,130,5586,80,1000,60,1000,Lausanne Dist,Lausanne Distribution,2,None,130,1998-03-18,100060,,VD
01,149,5586,30,1002,00,1000,Lausanne,Lausanne,2,None,149,1999-10-25,100200,,VD
01,151,5586,20,1004,00,1000,Lausanne,Lausanne,2,None,130,1986-05-21,100060,J,VD
01,153,5586,20,1006,00,1000,Lausanne,Lausanne,2,None,130,1986-05-21,100060,J,VD
01,154,5586,20,1007,00,1000,Lausanne,Lausanne,2,None,130,1986-05-21,100060,J,VD


### Converting PLZ_CSV as a Parquet Table with Partitions

1. We remove `Geo_Shape` and `Geokoordinaten`.
2. We use compression.
3. We use `Kanton` as a partition.

In [8]:
%%sql
CREATE TABLE IF NOT EXISTS plz (
    REC_ART VARCHAR(2),
    ONRP INT,
    BFSNR INT,
    PLZ_TYP SMALLINT,
    POSTLEITZAHL SMALLINT,
    PLZ_ZZ VARCHAR(2), 
    GPLZ SMALLINT,
    ORTBEZ18 VARCHAR(18),
    ORTBEZ27 VARCHAR(27),
    SPRACHCODE TINYINT,
    SPRACHCODE_ABW TINYINT,
    BRIEFZ_DURCH INT,
    GILT_AB_DAT DATE,
    PLZ_BRIEFZUST INT,
    PLZ_COFF VARCHAR(1)
)
PARTITIONED BY(KANTON string)
STORED AS Parquet
TBLPROPERTIES("parquet.compression"="SNAPPY")

 * hive://hadoop@localhost:10000/
Done.


[]

The partition `Kanton` needs to be the last column in the `insert` statement.

In [9]:
%%sql
INSERT INTO TABLE plz 
    SELECT
        REC_ART,
        ONRP,
        BFSNR,
        PLZ_TYP,
        POSTLEITZAHL,
        PLZ_ZZ, 
        GPLZ,
        ORTBEZ18,
        ORTBEZ27,
        SPRACHCODE,
        SPRACHCODE_ABW,
        BRIEFZ_DURCH,
        GILT_AB_DAT,
        PLZ_BRIEFZUST,
        PLZ_COFF,
        Kanton 
    FROM plz_csv

 * hive://hadoop@localhost:10000/
Done.


[]

In [86]:
%sql select * from plz limit 2

 * hive://hadoop@localhost:10000/
Done.


rec_art,onrp,bfsnr,plz_typ,postleitzahl,plz_zz,gplz,ortbez18,ortbez27,sprachcode,sprachcode_abw,briefz_durch,gilt_ab_dat,plz_briefzust,plz_coff,kanton
01,2597,4252,30,4303,03,4303,Kaiseraugst Lieb.,Kaiseraugst Liebrüti,1,None,2597,1997-03-05,430303,,AG
01,2600,4257,20,4305,00,4305,Olsberg,Olsberg,1,None,2604,1986-05-21,431200,J,AG


## Strassenbezeichnungen

In [15]:
!head -n5 /data/dataset/post/strassenbezeichnungen_v2.csv

REC_ART;STRID;ONRP;STRBEZK;STRBEZL;STRBEZ2K;STRBEZ2L;STR_LOK_TYP;STRBEZ_SPC;STRBEZ_COFF;STR_GANZFACH;STR_FACH_ONRP
04;15007157;1095;Torrent, place du;Torrent, place du;Place du Torrent;Place du Torrent;1;2;J;;
04;15007166;1095;Tronchet, en;Tronchet, en;En Tronchet;En Tronchet;1;2;J;;
04;15009148;1095;Versvey;Versvey;Versvey;Versvey;1;2;J;;
04;76147284;1096;Aiglon, route de l';Aiglon, route de l';Route de l'Aiglon;Route de l'Aiglon;1;2;J;;


In [17]:
%%sql
CREATE TABLE IF NOT EXISTS streets_csv (
    REC_ART VARCHAR(2),
    STRID INT,
    ONRP INT,
    STRBEZK VARCHAR(25),
    STRBEZL VARCHAR(60),
    STRBEZ2K VARCHAR(25),
    STRBEZ2L VARCHAR(60),
    STR_LOK_TYP TINYINT,
    STRBEZ_SPC TINYINT,
    STRBEZ_COFF VARCHAR(1),
    STR_GANZFACH VARCHAR(1),
    STR_FACH_ONRP INT
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ';'
LINES TERMINATED BY '\n'
STORED AS TEXTFILE
tblproperties ("skip.header.line.count"="1")

 * hive://hadoop@localhost:10000/
Done.


[]

In [23]:
%%sql
LOAD DATA LOCAL INPATH '/data/dataset/post/strassenbezeichnungen_v2.csv' INTO TABLE streets_csv

 * hive://hadoop@localhost:10000/
Done.


[]

In [28]:
%%sql
CREATE TABLE IF NOT EXISTS streets (
    REC_ART VARCHAR(2),
    STRID INT,
    ONRP INT,
    STRBEZK VARCHAR(25),
    STRBEZL VARCHAR(60),
    STRBEZ2K VARCHAR(25),
    STRBEZ2L VARCHAR(60),
    STR_LOK_TYP TINYINT,
    STRBEZ_SPC TINYINT,
    STRBEZ_COFF VARCHAR(1),
    STR_GANZFACH VARCHAR(1),
    STR_FACH_ONRP INT
)
STORED AS Parquet
TBLPROPERTIES("parquet.compression"="SNAPPY")

 * hive://hadoop@localhost:10000/
Done.


[]

In [29]:
%%sql
INSERT INTO TABLE streets 
    SELECT * FROM streets_csv

 * hive://hadoop@localhost:10000/
Done.


[]

In [31]:
%sql select * from streets limit 2

 * hive://hadoop@localhost:10000/
Done.


rec_art,strid,onrp,strbezk,strbezl,strbez2k,strbez2l,str_lok_typ,strbez_spc,strbez_coff,str_ganzfach,str_fach_onrp
04,15007157,1095,"Torrent, place du","Torrent, place du",Place du Torrent,Place du Torrent,1,2,J,,None
04,15007166,1095,"Tronchet, en","Tronchet, en",En Tronchet,En Tronchet,1,2,J,,None


## Bevölkerung

In [45]:
!head -n5 /data/dataset/post/bevoelkerung_proplz.csv

﻿Stichdatum;PLZ;Anzahl;Typ;ORTBEZ18
2022-03-01;8203;237;f;Schaffhausen
2022-03-01;8213;230;f;Neunkirch
2022-03-01;8239;567;m;Dörflingen
2022-03-01;8245;240;f;Feuerthalen


In [47]:
%%sql
CREATE TABLE IF NOT EXISTS bevoelkerung_csv (
    Stichdatum DATE,
    PLZ SMALLINT,
    ANZAHL INT,
    Typ VARCHAR(1),
    ORTBEZ18 VARCHAR(18)
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ';'
LINES TERMINATED BY '\n'
STORED AS TEXTFILE
tblproperties ("skip.header.line.count"="1")

 * hive://hadoop@localhost:10000/
Done.


[]

In [48]:
%%sql
LOAD DATA LOCAL INPATH '/data/dataset/post/bevoelkerung_proplz.csv' INTO TABLE bevoelkerung_csv

 * hive://hadoop@localhost:10000/
Done.


[]

In [50]:
%%sql
CREATE TABLE IF NOT EXISTS bevoelkerung (
    Stichdatum DATE,
    PLZ SMALLINT,
    ANZAHL INT,
    ORTBEZ18 VARCHAR(18)
)
PARTITIONED BY(Typ string)
STORED AS Parquet
TBLPROPERTIES("parquet.compression"="SNAPPY")

 * hive://hadoop@localhost:10000/
Done.


[]

In [54]:
%%sql
INSERT INTO TABLE bevoelkerung 
    SELECT Stichdatum, PLZ, ANZAHL, ORTBEZ18, TYP FROM bevoelkerung_csv

 * hive://hadoop@localhost:10000/
Done.


[]

In [55]:
%sql select * from bevoelkerung limit 3

 * hive://hadoop@localhost:10000/
Done.


stichdatum,plz,anzahl,ortbez18,typ
2022-03-01,8203,237,Schaffhausen,f
2022-03-01,8213,230,Neunkirch,f
2022-03-01,8245,240,Feuerthalen,f


## Nachnamen

In [57]:
!head -n5 /data/dataset/post/nachnamen_proplz.csv

﻿Stichdatum;PLZ;Geschlecht;Nachname;Anzahl;Rang;ORTBEZ18
2022-03-01;2905;m;Gerber;14;1;Courtedoux
2022-03-01;2905;m;Fleury;7;5;Courtedoux
2022-03-01;2906;m;Laville;34;1;Chevenez
2022-03-01;2906;m;Borruat;19;4;Chevenez


In [60]:
%%sql
CREATE TABLE IF NOT EXISTS nachnamen_csv (
    Stichdatum DATE,
    PLZ SMALLINT,
    Geschlecht VARCHAR(1),
    Nachname STRING,
    Anzahl INT,
    Rang TINYINT,
    ORTBEZ18 VARCHAR(18)
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ';'
LINES TERMINATED BY '\n'
STORED AS TEXTFILE
tblproperties ("skip.header.line.count"="1")

 * hive://hadoop@localhost:10000/
Done.


[]

In [61]:
%%sql
LOAD DATA LOCAL INPATH '/data/dataset/post/nachnamen_proplz.csv' INTO TABLE nachnamen_csv

 * hive://hadoop@localhost:10000/
Done.


[]

In [62]:
%%sql
CREATE TABLE IF NOT EXISTS nachnamen (
    Stichdatum DATE,
    PLZ SMALLINT,
    Nachname STRING,
    Anzahl INT,
    Rang TINYINT,
    ORTBEZ18 VARCHAR(18)
)
PARTITIONED BY(Geschlecht string)
STORED AS Parquet
TBLPROPERTIES("parquet.compression"="SNAPPY")

 * hive://hadoop@localhost:10000/
Done.


[]

In [63]:
%%sql
INSERT INTO TABLE nachnamen 
    SELECT Stichdatum, PLZ, Nachname, Anzahl, Rang, ORTBEZ18, Geschlecht FROM nachnamen_csv

 * hive://hadoop@localhost:10000/
Done.


[]

In [64]:
%sql select * from nachnamen limit 3

 * hive://hadoop@localhost:10000/
Done.


stichdatum,plz,nachname,anzahl,rang,ortbez18,geschlecht
2022-03-01,2905,Gerber,14,1,Courtedoux,m
2022-03-01,2905,Fleury,7,5,Courtedoux,m
2022-03-01,2906,Laville,34,1,Chevenez,m
